# Install requirements



In [72]:
pip install tenseal sentence_transformers

In [73]:
import tenseal as ts # pip install tenseal
import base64
from sentence_transformers import SentenceTransformer

#Setup Embedding Model

In [74]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [75]:
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "I watched the sunset over the ocean.",
    "Artificial intelligence is transforming industries.",
    "The library was quiet and smelled like old books.",
    "He dreamed of traveling to distant galaxies.",
    "Innovation drives progress in the tech world.",
    "The chef prepared a delicious meal for the guests.",
    "Climate change poses a significant threat to global biodiversity.",
    "The athlete trained rigorously for the upcoming marathon.",
    "Music has the power to evoke deep emotional responses."
]

# Get and Store Sentence Embeddings

In [76]:
embeddings = model.encode(sentences)
print(embeddings)

[[ 0.04393356  0.05893442  0.04817836 ...  0.05216276  0.05610649
   0.10206394]
 [ 0.02740657  0.0430141   0.03891347 ... -0.01506037 -0.08190976
   0.03566786]
 [ 0.03608515 -0.02447893  0.05895455 ... -0.08262778  0.07626031
  -0.02810953]
 ...
 [-0.04316908  0.12324493  0.09774672 ... -0.13736206 -0.0542914
  -0.02212726]
 [-0.01195203  0.08340716 -0.03579247 ... -0.0014969   0.00464318
  -0.00069307]
 [ 0.05495496 -0.0110149   0.09608352 ...  0.05075204  0.01033213
  -0.03917952]]


# Get Query Embedding

In [137]:
query_text = "brown"
query_embedding = model.encode([query_text])
print(query_embedding)

[[-1.05881561e-02  2.85556484e-02 -4.00093421e-02  1.10943960e-02
   2.13167034e-02 -2.79458780e-02  1.40698910e-01 -6.80966973e-02
   3.41516398e-02  1.86501723e-02 -7.26412162e-02 -1.02453955e-01
  -3.91524881e-02 -4.99941446e-02  1.07298223e-02  5.85493147e-02
   3.73814180e-02 -2.63625607e-02 -8.92162994e-02 -1.00236043e-01
  -8.81764293e-02  4.41198796e-02 -3.31310555e-02  1.93479806e-02
  -7.97645748e-02  4.61917967e-02  2.58189160e-02 -2.61937920e-02
  -3.91580351e-02 -1.38522133e-01 -5.02741858e-02 -8.80908445e-02
   2.97451839e-02 -2.53110677e-02 -5.10086156e-02 -8.96003023e-02
  -4.55906577e-02 -5.00377687e-03 -4.39924188e-02 -8.37920140e-03
  -2.78143510e-02 -8.71248320e-02 -4.60331142e-02  9.33047757e-02
   3.77245271e-03  5.33336513e-02  4.66135889e-03  4.65625413e-02
  -4.58975732e-02  5.34727499e-02  1.08526107e-02 -2.74228789e-02
  -6.68582991e-02  1.26600983e-02  1.75823662e-02  3.23846266e-02
  -2.78384611e-02  1.45997033e-02  7.68656284e-02 -5.25192916e-02
   2.11349

# Setup CKKS Homomorphic Encryption

In [138]:
context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree = 8192,
            coeff_mod_bit_sizes = [60, 40, 40, 60]
          )

context.generate_galois_keys()
context.global_scale = 2**40

In [139]:
secret_context = context.serialize(save_secret_key = True)

In [140]:
context.make_context_public() #drop the secret_key from the context
public_context = context.serialize()

In [141]:
context = ts.context_from(secret_context)

# Encrypt All Embeddings

In [142]:
enc_queryvec = ts.ckks_vector(context, query_embedding[0].tolist())
print(enc_queryvec.decrypt())

[-0.010588156065170336, 0.02855564979306049, -0.040009343566382215, 0.011094395619597302, 0.021316702332205142, -0.027945878055640687, 0.1406989093118751, -0.06809669654560527, 0.034151639100655044, 0.018650172403258728, -0.07264121633695347, -0.10245395451242453, -0.039152487891927344, -0.049994142263062466, 0.010729823211542776, 0.05854931526002381, 0.03738141667274147, -0.026362560853831103, -0.08921629987929683, -0.10023604229753721, -0.08817642941283625, 0.044119876611273784, -0.03313105495183437, 0.019347978730851582, -0.07976457473655554, 0.04619179465114802, 0.025818915961922678, -0.02619379194704857, -0.03915803367978495, -0.13852213301454128, -0.05027418517428894, -0.0880908436821496, 0.029745181649502583, -0.02531106652816352, -0.051008615701335205, -0.0896003006679342, -0.04559065779935685, -0.005003777820824271, -0.0439924189463748, -0.00837920195764532, -0.027814347054183383, -0.08712483184385565, -0.04603311347066353, 0.0933047777847231, 0.0037724549700822382, 0.05333364

Measure Eucledian Distance

In [145]:
euclidean_difference_ranking = []
for i in range (len(sentences)):
  enc_sentence = ts.ckks_vector(context, embeddings[i].tolist())
  euclidean_squared =   enc_queryvec - enc_sentence
  euclidean_squared = euclidean_squared.dot(euclidean_squared)
  euclidean_difference_ranking.append({sentences[i]:abs(euclidean_squared.decrypt()[0])})

# Present Results

In [146]:
search_results = sorted(euclidean_difference_ranking, key=lambda x: list(x.values())[0])
for item in search_results:
  print(item)
  print()

{'The quick brown fox jumps over the lazy dog.': 1.51384260308229}

{'The athlete trained rigorously for the upcoming marathon.': 1.7591404798698158}

{'The chef prepared a delicious meal for the guests.': 1.7646651318017312}

{'I watched the sunset over the ocean.': 1.8184784707135617}

{'The library was quiet and smelled like old books.': 1.8491573796139107}

{'He dreamed of traveling to distant galaxies.': 1.8656390952538908}

{'Artificial intelligence is transforming industries.': 1.9501590603775918}

{'Climate change poses a significant threat to global biodiversity.': 1.9566377585547878}

{'Music has the power to evoke deep emotional responses.': 1.9925434698952273}

{'Innovation drives progress in the tech world.': 2.0264233588357863}

